In [4]:
# Assumptions are as follows
# Computer is always X
# Computer is always the maximizer
# Human is always O
# Human is always the minimizer
# Human always goes first

from math import inf as infinity
from random import choice
import copy

In [5]:
class State:
    def __init__ (self,board,score):
        self.score = score
        self.board = board



def printBoard(b):
    for row in (b):
        for num in row:
            if num == 0:
                print('_   ', end=' ')
            elif num == 1:
                print('X   ', end=' ')
            elif num == -1:
                print('O   ', end=' ')
        print('\n')

# Check all types of wins
def checkWin(board):
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != 0:
            if (board[i][0] == 1):
                return 'X'
            else:
                return 'O'
        elif board[0][i] == board[1][i] == board[2][i] != 0:
            if (board[0][i] == 1):
                return 'X'
            else:
                return 'O'
    if board[0][0] == board[1][1] == board[2][2] != 0:
            if (board[0][0] == 1):
                return 'X'
            else:
                return 'O'
    elif board[0][2] == board[1][1] == board[2][0] != 0:
            if (board[0][2] == 1):
                return 'X'
            else:
                return 'O'
    if (0 not in board[0] and 0 not in board[1] and 0 not in board[2]):
        return 'tie'
    return 0

def isLeaf(board):
    if checkWin(board) != 0:
        return True
    return False

# Create all states for some given depth
def createStates(board,player):
    states = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0: # For empty spaces in the board
                tempBoard = copy.deepcopy(board)
                tempBoard[i][j] = (1 if player == 'X' else -1) # If player is X (Computer), then the value is 1, else -1 (Human)
                # Evaluate the board if it is the last depth else set the score to infinity or -infinity depending on the Minima or Maxima
                if isLeaf(tempBoard):
                    states.append(State(tempBoard,evaluate(tempBoard)))
                else:
                    states.append(State(tempBoard,(-infinity if player == 'X' else infinity)))
                    
    return states

def evaluate(board):
    if checkWin(board) == 'X':
        return 10
    elif checkWin(board) == 'O':
        return -10
    elif checkWin(board) == 'tie':
        return 0

# Min max algorithm
# Computer turn maximizes
# Human turn minimizes
# Recursivly call minmax for the next depth
# Last deapth is the leaf node which cant be more than 9 total depth
def minmax(state,player,alpha,beta):
    # If last move or game is over, return the state
    # If final depth where score is set to evaluate(board) is reached, return the state
    if isLeaf(state.board):
        return state # Return single state which already has the score set as its the leaf node. Refer to evaluate function
    
    # Create all possible states
    states = createStates(state.board,player)
    # If Computer playing then find the maximum score for the depth
    if player == 'X':
        best = -infinity
        # For all states, find the maximum score
        for s in states:
            val = minmax(s,'O',alpha,beta) # Call minmax for the next depth and player
            s.score = val.score # Set the score of the state to the score of the returned state
            best = max(best,s.score) # Find the maximum score
            alpha = max(alpha,best) # Set alpha to the maximum score
            if beta <= alpha: # If beta is less than or equal to alpha, then prune the branch
                break
        for s in states: # Return the state with the maximum score
            if s.score == best: # If the score of the state is the maximum score
                return s # Return the best state
    else:
        best = infinity
        for s in states:
            val = minmax(s,'X',alpha,beta)
            s.score = val.score
            best = min(best,s.score)

            beta = min(beta,best)
            if beta <= alpha:
                break
            
        for s in states:
            if s.score == best:
                return s # Return the best state

# Alpha Beta are like global best and worst scores
# Alpha is the best score for the maximizer
# Beta is the best score for the minimizer
# If score of state is bad acording to alpha or beta, then prune the branch (Dont explore)
# This increases the speed of the algorithm
def alphaBetaPruning(state,player):
    return minmax(state,player,-infinity,infinity)

def humanTurn(myGlobalState):
    while True:
        try:
            x = int(input('Enter x: '))
            y = int(input('Enter y: '))
            if myGlobalState.board[x][y] == 0:
                myGlobalState.board[x][y] = -1
                myGlobalState.score = infinity
                break
            else:
                print('Invalid move')
        except:
            print('Invalid move')

In [6]:
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
]

myGlobalState = State(board,infinity)
humanTurn(myGlobalState)
while True:
    printBoard(myGlobalState.board)
    print("\n======================\n")
    if checkWin(myGlobalState.board) != 0:
        break
    myGlobalState = alphaBetaPruning(myGlobalState,'X') # Call minmax for computers turn
    #print("Score is ", myGlobalState.score)
    printBoard(myGlobalState.board)
    print("\n======================\n")
    if checkWin(myGlobalState.board) != 0:
        break
    humanTurn(myGlobalState)

print(checkWin(myGlobalState.board), " Won the game") if checkWin(myGlobalState.board) != 'tie' else print("Game Tied !")



O    _    _    

_    _    _    

_    _    _    



O    _    _    

_    X    _    

_    _    _    



Invalid move
Invalid move
O    _    _    

_    X    _    

_    O    _    



O    _    _    

X    X    _    

_    O    _    



O    _    _    

X    X    O    

_    O    _    



O    _    X    

X    X    O    

_    O    _    



O    O    X    

X    X    O    

_    O    _    



O    O    X    

X    X    O    

X    O    _    



X  Won the game
